# Criando um modelo de pegar captura de modo Holístico, do corpo todo
# O modelo rastrea o total de 543 pontos no corpo humano, sendo 
# 33 da pose, como o modelo de pose
# 21 pontos por mão, com 42 no total, com o modelo de mãos
# 468 para o rosto, pelo modelo de face mesh

In [1]:
# Importando as bibliotecas
import cv2
import mediapipe as mp

In [2]:
# Definindo as variáveis
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
frames_error = 0

In [4]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    success, image = cap.read()

    if not success:
      print("Ignorando imagem vazia")
      continue

    if frames_error > 10:
      print("Sem sucesso ao capturar o vídeo")
      break

    # Converter cores para RGB (requerido pelo MediaPipe)
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Desenhando as landmarks na imagem, para isso, voltamos para o BGR para o OpenCV
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Esse a seguir, serve para mostrar o rosto, mas não seguirei nele

    # mp_drawing.draw_landmarks(
    #    image,
    #    results.face_landmarks,
    #    mp_holistic.FACEMESH_CONTOURS,
    #    landmark_drawing_spec=None,
    #    connection_drawing_spec=mp_drawing_styles
    #    .get_default_face_mesh_contours_style())
    
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    
    if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.right_hand_landmarks,
            mp_holistic.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style()
        )

    cv2.imshow('Modelo Holístico do MediaPipe', cv2.flip(image, 1))

    if cv2.waitKey(5) & 0xFF == 27:
      break

cap.release()